<a href="https://colab.research.google.com/github/MatteoGuglielmi-tech/Polarity-and-Subjectivity-Detection/blob/main/NTN_no_print.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from typing import Tuple, List
import numpy as np
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
class NeuralTensorNetwork(nn.Module):
    def __init__(self, output_dim: int, input_dim: int, activation: str="tanh", mean: float=0.0, std: float=1.0):
        
        super(NeuralTensorNetwork, self).__init__()

        # setting input and output dimensions
        self.k = output_dim
        self.d = input_dim # e1,e2

        # setting mean and std for random initialization
        self.mean = mean
        self.std = std

        self.activation = activation

        # parameters has been used in order to consider W, V, b as model parameters
        # inference -> they'll be optimized

        # normal sampling -> https://pytorch.org/docs/stable/generated/torch.normal.html
        # parameter -> https://pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html#torch.nn.parameter.Parameter
        self.W = nn.Parameter(torch.normal(self.mean, self.std, size=(self.k, self.d, self.d)))
        self.V = nn.Parameter(torch.normal(self.mean, self.std, size=(2*self.d, self.k)))
        self.b = nn.Parameter(torch.zeros(size=(self.d,)))
        
        #print(f"self.W : {torch.Tensor.size(self.W)}")
        #print(f"self.V : {torch.Tensor.size(self.V)}")
        #print(f"self.b : {torch.Tensor.size(self.b)}")
        
        if activation == 'tanh':
            self.activation = nn.Tanh()
        elif activation == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif self.activation == 'relu':
            self.activation = nn.ReLU()
        # checking for a good activation function
        else:
            raise ValueError('Possible activation choices are tanh, sigmoid or ReLU')

    def forward(self, inputs: List[torch.Tensor]) -> torch.Tensor:

        # getting the entities
        e1 = inputs[0]
        e2 = inputs[1]
        #print(f"e1.shape : {torch.Tensor.size(e1)}")
        #print(f"e2.shape : {torch.Tensor.size(e2)}")
        #print(f"self.W[0] : {torch.Tensor.size(self.W[0])}")
        #print(f"self.V : {torch.Tensor.size(self.V)}")
        #print(f"self.bias : {torch.Tensor.size(self.b)}")
        #print(f"batch_size : {torch.clone(e1).cpu().numpy().shape[0]}")
        #print(f"k : {self.k}")
        #print(f"torch.cat([e1,e2], dim=1) : {torch.Tensor.size(torch.cat([e1,e2], dim=1))}")
        

        # input tensor should be of shape (batch_size, padded_length, 768)
        batch_size = torch.clone(e1).cpu().numpy().shape[0]
        k = self.k
        d = self.d

        # print(f"dot prod : {torch.matmul(e1, self.W[0])}")
        #print(f"dot prod size: {torch.Tensor.size(torch.matmul(e1, self.W[0]))}")

        # bilinear tensor + bias
        bil_bias = [torch.sum((e2 * torch.matmul(e1, self.W[0])) + self.b, axis=1)]
        
        for i in range(1,k):
            bil_bias.append(torch.sum((e2*torch.matmul(e1, self.W[i])) + self.b, axis=1))
        
        bil_bias = torch.cat(bil_bias, axis=0)
        bil_bias = torch.reshape(bil_bias, (batch_size, k))

        # Vr * [e1, e2]
        rest = torch.matmul(torch.cat([e1,e2], dim=1), self.V)

        e1_R_e2 = bil_bias + rest

        # applying activation
        f = self.activation(e1_R_e2)
        return f